# Week 5 Assignment

Import modules and prep data for analysis.

In [10]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split

import timeit

import warnings
warnings.filterwarnings("ignore")

In [4]:
df = pd.read_csv('../prepped_churn_data.csv', index_col='customerID')
df

,tenure,PhoneService,Contract,PaymentMethod,MonthlyCharges,TotalCharges,Churn,zscore_tenure,zscore_monthlyCharge,zscore_totalCharge,avgCharges,chargeDiff
customerID,,,,,,,,,,,,
7590-VHVEG,1,0,0,1,29.85,29.85,0,1.277445,1.160323,0.992667,29.850000,0.000000
5575-GNVDE,34,1,1,0,56.95,1889.50,0,0.066327,0.259629,0.172198,55.573529,-1.376471
3668-QPYBK,2,1,0,0,53.85,108.15,1,1.236724,0.362660,0.958122,54.075000,0.225000
7795-CFOCW,45,0,1,2,42.30,1840.75,0,0.514251,0.746535,0.193706,40.905556,-1.394444
9237-HQITU,2,1,0,1,70.70,151.65,1,1.236724,0.197365,0.938930,75.825000,5.125000
...,...,...,...,...,...,...,...,...,...,...,...,...
6840-RESVB,24,1,1,0,84.80,1990.50,0,0.340876,0.665992,0.127637,82.937500,-1.862500
2234-XADUH,72,1,1,3,103.20,7362.90,0,1.613701,1.277533,2.242641,102.262500,-0.937500
4801-JZAZL,11,0,0,1,29.60,346.45,0,0.870241,1.168632,0.852985,31.495455,1.895455


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7043 entries, 7590-VHVEG to 3186-AJIEK
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   tenure                7043 non-null   int64  
 1   PhoneService          7043 non-null   int64  
 2   Contract              7043 non-null   int64  
 3   PaymentMethod         7043 non-null   int64  
 4   MonthlyCharges        7043 non-null   float64
 5   TotalCharges          7043 non-null   float64
 6   Churn                 7043 non-null   int64  
 7   zscore_tenure         7043 non-null   float64
 8   zscore_monthlyCharge  7043 non-null   float64
 9   zscore_totalCharge    7043 non-null   float64
 10  avgCharges            7043 non-null   float64
 11  chargeDiff            7043 non-null   float64
dtypes: float64(7), int64(5)
memory usage: 715.3+ KB


Create features and targets from data for ML models

In [7]:
features = df.drop(['Churn'],axis=1)
targets = df['Churn']

x_train, x_test, y_train, y_test = train_test_split(features, targets, random_state=42)

## Use tpot to find an ML algorithm that performs best on the data

TPOT was used and found that the best algorithm is the ExtraTreesClassifier

In [12]:
%%time
tpot = TPOTClassifier(generations=5, population_size=50, cv=5,random_state=42, scoring='accuracy', verbosity=2, n_jobs=-1)

tpot.fit(x_train, y_train)
print(tpot.score(x_test, y_test))

Optimization Progress:   0%|          | 0/300 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.7960970872394715

Generation 2 - Current best internal CV score: 0.796097445600757

Generation 3 - Current best internal CV score: 0.796475337576331

Generation 4 - Current best internal CV score: 0.7966658065995814

Generation 5 - Current best internal CV score: 0.7983701728734842

Best pipeline: ExtraTreesClassifier(SGDClassifier(input_matrix, alpha=0.001, eta0=0.01, fit_intercept=True, l1_ratio=0.75, learning_rate=constant, loss=hinge, penalty=elasticnet, power_t=100.0), bootstrap=True, criterion=gini, max_features=0.35000000000000003, min_samples_leaf=5, min_samples_split=12, n_estimators=100)
0.8018171493469619
CPU times: total: 60 s
Wall time: 2min 37s


We can see that the model gave us an 80% accuracy.

In [13]:
predictions = tpot.predict(x_test)
predictions

array([1, 0, 0, ..., 0, 1, 0], dtype=int64)

In [14]:
# display the actuals and predictions for the test set
print('Predictions for test data set')
print(predictions)
print('Actuals for test data set')
print(y_test)

from sklearn.metrics import accuracy_score
print(f'Accuracy of the TPOT predictions: {accuracy_score(y_test,predictions)}')

Predictions for test data set
[1 0 0 ... 0 1 0]
Actuals for test data set
customerID
1024-GUALD    1
0484-JPBRU    0
3620-EHIMZ    0
6910-HADCM    1
8587-XYZSF    0
             ..
4581-SSPWD    1
4813-HQMGZ    0
4903-CNOZC    0
0822-GAVAP    1
3055-OYMSE    0
Name: Churn, Length: 1761, dtype: int64
Accuracy of the TPOT predictions: 0.8018171493469619


## Save the model to disk

In [15]:
tpot.export('tpot_churn_pipeline.py')

## Create a Python script/file/module with a function that takes a pandas dataframe as an input and returns the probability of churn for each row in the dataframe

your Python file/function should print out the predictions for new data (new_churn_data.csv) the true values for the new data are [1, 0, 0, 1, 0] if you're interested

I created a script from the exported TPOT pipeline to predict the new data. As seen below, the prediction model correctly predicted 4/5, or 80%, which is exactly what is expected based on the test data accuracy.


In [28]:
%run tpot_churn_pipeline.py

[1 0 0 0 0]


## Summary

Overall, the model chosen by TPOT gave us an accuracy of 80% based on the test data. When we ran the model against the new data, it also returned an 80% accuracy, so it performed as expected. Given more time, TPOT may be able to create a model with a higher accuracy.